In [5]:
import os
import pandas as pd
import numpy as np
import nibabel as nb
import math as m
import pickle
from tkinter import Tk
from tkinter.filedialog import askopenfilename
from compile_studies import compile_studies


cwd = os.getcwd()
raw_folder = cwd + '/DataRaw/'
pickle_folder = cwd + '/DataPickle/'
mask_folder = cwd + '/MaskenEtc/'

'''filename = askopenfilename()
df = pd.read_excel(filename, engine='openpyxl', header=None)'''

df = pd.read_excel(raw_folder + 'Metastocalculate.xlsx', engine='openpyxl', header=None)
with open(pickle_folder + 'data.pickle', 'rb') as f:
    experiments, tasks = pickle.load(f)

In [6]:
for index, row in df.iterrows():
    if row[0] == 'M': #Main Effect Analysis
        s0 = compile_studies(df, index, experiments, tasks)
        if len(s0) >= 12:
            category = df.iloc[index, 1]
            mean_subjects = experiments.iloc[s0].Subjects.mean()
            print(category + ' : ' + str(len(s0)) + 'experiments; average of {mean} subjects per experiment'.format(mean=mean_subjects))

Other : 19experiments; average of 22.263157894736842 subjects per experiment
Cwclassic : 37experiments; average of 25.756756756756758 subjects per experiment
Cwvariant : 21experiments; average of 26.333333333333332 subjects per experiment
CwIvsC : 58experiments; average of 25.96551724137931 subjects per experiment
CwIvsN : 41experiments; average of 22.21951219512195 subjects per experiment
CwIvsNwords : 17experiments; average of 22.235294117647058 subjects per experiment
CwIvsNsymbolsletters : 23experiments; average of 22.304347826086957 subjects per experiment


### ComputeALEtfce

In [5]:
template = nb.load(mask_folder + "Grey10.nii")
template_shape = template.get_fdata().shape

In [6]:
prior = np.zeros(template.shape)
uc = 0.001
c = str(uc)[2:]

In [7]:
folders_req = ['Volumes', 'NullDistributions', 'VolumesZ', 'VolumesTFCE', 'Results', 'Images', 'Foci']
folders_req_imgs = ['Foci', 'ALE', 'TFCE']

os.mkdir('ALE')
for folder in folders_req:
    print(os.getcwd())
    if folder == 'Images':
        for sub_folder in folders_req_imgs:
            os.mkdir('ALE/Images/' + sub_folder)

/Users/lfrahm/IDrive-Sync/sync_Documents/PhD/Code/ALE/pyALE
/Users/lfrahm/IDrive-Sync/sync_Documents/PhD/Code/ALE/pyALE
/Users/lfrahm/IDrive-Sync/sync_Documents/PhD/Code/ALE/pyALE
/Users/lfrahm/IDrive-Sync/sync_Documents/PhD/Code/ALE/pyALE
/Users/lfrahm/IDrive-Sync/sync_Documents/PhD/Code/ALE/pyALE
/Users/lfrahm/IDrive-Sync/sync_Documents/PhD/Code/ALE/pyALE
/Users/lfrahm/IDrive-Sync/sync_Documents/PhD/Code/ALE/pyALE


In [8]:
S

,Author,Subjects,Space,Cond,XYZmm,UncertainTemplates,UncertainSubjects,Smoothing,XYZ,Kernel,Peaks
0,"Critchley et al., 2005",15,MNI,"[i>c, visual, numerical, other, masked, fmri]","[[12.0, 28.0, 58.0, 54.0, 6.0], [18.0, 2.0, 14...",8.411288,4.419774,9.501798,"[[39.0, 31.0, 16.0, 18.0, 42.0], [72.0, 64.0, ...",NaN,5
1,"Kerns et al., 2005",13,TAL,"[i>c, visual, colorword, cw, unmasked, fmri, m...","[[2.0, -28.0, 44.0, -37.0, 42.0, 56.0, -58.0, ...",8.411288,4.747598,9.658647,"[[44.0, 59.0, 23.0, 63.5, 24.0, 17.0, 74.0, 25...",NaN,13
2,"Bush, 1998",9,TAL,"[i>nrb, neutral, visual, counting, other, unma...","[[43.0, -46.0, 12.0, -56.0, -40.0, 43.0, -28.0...",8.411288,5.705903,10.164010,"[[23.5, 68.0, 39.0, 73.0, 65.0, 23.5, 59.0], [...",NaN,7
3,"Pardo et al., 1990",8,TAL,"[i>c, visual, colorword, cw, unmasked, pet, no...","[[10.0, 7.0, 17.0, -27.0, -25.0, -41.0, -17.0,...",8.411288,6.052024,10.362276,"[[40.0, 41.5, 36.5, 58.5, 57.5, 65.5, 53.5, 57...",NaN,13
4,"Bench et al., 1993",6,TAL,"[i>nrb, neutral, visual, colorword, cw, unmask...","[[-32.0, 38.0], [-64.0, -44.0], [44.0, 48.0]]",8.411288,6.988276,10.935528,"[[61.0, 26.0], [31.0, 41.0], [58.0, 60.0], [1....",NaN,2
...,...,...,...,...,...,...,...,...,...,...,...
120,Peven et al. 2019,50,MNI,"[i>nrb, neutral, visual, colorword, cw, unmask...","[[36.0, 56.0, 46.0, 30.0, 42.0, 42.0, -46.0, -...",8.411288,2.420810,8.752719,"[[27.0, 17.0, 22.0, 30.0, 24.0, 24.0, 68.0, 70...",NaN,30
121,Kozasa et al. 2018,33,MNI,"[i>c, visual, colorword, cw, unmasked, fmri, m...","[[-42.0, -8.0, -30.0, -38.0, 0.0, -22.0, 0.0, ...",8.411288,2.979811,8.923511,"[[66.0, 49.0, 60.0, 64.0, 45.0, 56.0, 45.0, 57...",NaN,10
122,Wagner et al. 2016,40,MNI,"[i>c, visual, colorword variant, cw, unmasked,...","[[-30.0, -36.0, -52.0, -46.0, -48.0, -44.0, -4...",8.411288,2.706548,8.836016,"[[60.0, 63.0, 71.0, 68.0, 69.0, 67.0, 67.0, 65...",NaN,30
123,Ghavidel et al. 2020,18,MNI,"[i>c, visual, colorword, cw, unmasked, fmri, m...","[[-8.0, -6.0, -8.0, -42.0, -50.0, -4.0], [20.0...",8.411288,4.034683,9.328903,"[[49.0, 48.0, 49.0, 66.0, 70.0, 47.0], [73.0, ...",NaN,6
